In [ ]:
# Ch14-2-boto3

In [ ]:
# Install boto3 

In [ ]:
! pip install boto3

In [ ]:
## Build an S3 bucket ##

In [ ]:
# Import Libraries

In [ ]:
import boto3
from botocore.exceptions import ClientError

In [ ]:
# Function to create and S3 bucket using boto3
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region."""
    try:
        s3_client = boto3.client('s3', region_name=region)      
        if region is None:
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        print(f"Bucket '{bucket_name}' created successfully.")
    except ClientError as e:
        print(f"Error: {e}")
        return False
    return True

In [ ]:
# Set up versioining on the bucket 
def enable_versioning(bucket_name):
    """Enable versioning on the S3 bucket."""
    s3 = boto3.client('s3')
    versioning_config = {
        'Status': 'Enabled'
    }
    s3.put_bucket_versioning(Bucket=bucket_name, VersioningConfiguration=versioning_config)
    print(f"Versioning enabled on bucket '{bucket_name}'.")

In [ ]:
def upload_sample_file(bucket_name, file_name, object_name=None):
    """Upload a sample file to the S3 bucket."""
    if object_name is None:
        object_name = file_name

    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_name, bucket_name, object_name)
        print(f"File '{file_name}' uploaded to '{bucket_name}/{object_name}'.")
    except ClientError as e:
        print(f"Upload error: {e}")

In [ ]:
# ---- Main Deployment Logic ----
bucket = "sab-bucket-4"  # Replace with a globally unique name of your choosing
region = "us-west-2"  # Choose your region

# 1. Create the bucket
if create_bucket(bucket, region):
    # 2. Enable versioning
    enable_versioning(bucket)
    
    # 3. Upload a sample file (optional)
    with open("sample.txt", "w") as f:
        f.write("This is a sample file.")
    upload_sample_file(bucket, "sample.txt")

In [ ]:
## Delete the S3 bucket ##

In [ ]:
def empty_bucket(bucket_name):
    """Delete all objects and object versions from the bucket."""
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)

    try:
        # Delete all object versions (for versioned buckets)
        bucket.object_versions.delete()
        print(f"All objects and versions deleted from '{bucket_name}'.")
    except ClientError as e:
        print(f"Error emptying bucket: {e}")
        return False
    return True

In [ ]:
def delete_bucket(bucket_name):
    """Delete the S3 bucket after it has been emptied."""
    s3 = boto3.client('s3')
    try:
        s3.delete_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' deleted successfully.")
    except ClientError as e:
        print(f"Error deleting bucket: {e}")
        return False
    return True

In [ ]:
# ---- Main Execution ----
bucket_to_delete = "sab-bucket-4"  # Replace with your bucket name

# 1. Empty the bucket
if empty_bucket(bucket_to_delete):
    # 2. Delete the bucket
    delete_bucket(bucket_to_delete)

In [ ]:
### Part 2 - CloudFormation Templates ###

In [ ]:
# Build an S3 bucket using CloudFormation
#. First make the s3-bucket.yaml file

In [ ]:
# Run this command:
aws cloudformation create-stack \
  --stack-name my-s3-stack \
  --template-body file://s3-bucket.yaml \
  --capabilities CAPABILITY_NAMED_IAM

In [ ]:
# Delete the CloudFormation stack
aws cloudformation delete-stack --stack-name my-s3-stack

In [ ]:
## End of Notebook ##